In [3]:
from langchain_groq import ChatGroq
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.utilities import SerpAPIWrapper
import os

In [ ]:
os.environ["GROQ_API_KEY"] ="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"

In [20]:
import os
os.environ["GROQ_API_KEY"] = "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"

from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model_name="deepseek-r1-distill-llama-70b"
)


In [21]:
llm.invoke("hello")

AIMessage(content='<think>\n\n</think>\n\nHello! How can I assist you today? 😊', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 4, 'total_tokens': 20, 'completion_time': 0.074313814, 'prompt_time': 5.208e-05, 'queue_time': 0.0591291, 'total_time': 0.074365894}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_1bbe7845ec', 'finish_reason': 'stop', 'logprobs': None}, id='run--e0729188-6074-48ee-aa12-000cdc06350a-0', usage_metadata={'input_tokens': 4, 'output_tokens': 16, 'total_tokens': 20})

In [8]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

docs = PyMuPDFLoader("https://arxiv.org/pdf/2404.19553").load()
#
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 0,
    length_function = len,
)

split_chunks = text_splitter.split_documents(docs)

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
EMBEDDING_MODEL_NAME = "thenlper/gte-small"
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}, 
)

/home/sahil/miniconda3/envs/py312mlops/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
from langchain_community.vectorstores import Qdrant

qdrant_vectorstore = Qdrant.from_documents(
    split_chunks,
    embedding_model,
    location=":memory:",
    collection_name="extending_context_window_llama_3",
)

/home/sahil/miniconda3/envs/py312mlops/lib/python3.12/site-packages/qdrant_client/http/models/models.py:758: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is empty, alternative syntax for `is_empty: \&quot;field_name\&quot;`",
/home/sahil/miniconda3/envs/py312mlops/lib/python3.12/site-packages/qdrant_client/http/models/models.py:762: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is null, alternative syntax for `is_null: \&quot;field_name\&quot;`",


In [ ]:
qdrant_retriever_sport = qdrant_vectorstore.as_retriever()
qdrant_retriever_movie = qdrant_vectorstore.as_retriever()
qdrant_retriever_stock = qdrant_vectorstore.as_retriever()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser

RAG_PROMPT = """

CONTEXT:
{context}

QUERY:
{question}

You are a helpful assistant. Use the available context to answer the question. If you can't answer the question, say you don't know.
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

#

rag_chain = (
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)
#
rag_chain.invoke({"question" : "What does the 'context' in 'long context' refer to give me answer in sortest way?"})


"<think>\nOkay, so I need to figure out what the 'context' in 'long context' refers to based on the provided documents. Let me start by reading through the snippets given.\n\nIn the first document, it mentions two types of long context: homogeneous and heterogeneous. Homogeneous context is described as a coherent text, like a book or a long paper. So, that suggests that context here refers to a body of text that's lengthy and cohesive.\n\nThe second document talks about the length of context being between 64K to 80K tokens. It also mentions organizing question-answer pairs in one multi-turn conversation. This implies that the context is the information provided to the model to answer questions, and in this case, it's a large amount of text.\n\nThe third and fourth documents discuss evaluations on LongBench, which has real-world long-context tasks. They mention using 32K context length by default and 8K for a specific model. This further supports that context refers to the text length t

In [ ]:
from typing import Annotated, List
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from typing_extensions import TypedDict
from langchain.chat_models import ChatOpenAI
from langchain_core.tools import tool


def run_rag_multidomain_agent(user_query: str) -> str:
    # ---- Step 1: Define state ----
    class State(TypedDict):
        messages: Annotated[List[BaseMessage], add_messages]

    # ---- Step 2: Define LLM ----
    llm = ChatOpenAI(temperature=0)

    # ---- Step 3: Define Tools ----
    @tool
    def medical_expert(question: str) -> str:
        """Answer medical-related questions."""

        RAG_PROMPT = """

            CONTEXT:
            {context}

            QUERY:
            {question}

            You are a helpful assistant. Use the available context to answer the question. If you can't answer the question, say you don't know.
            """


        rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

        #

        rag_chain = (
            {"context": itemgetter("question") | qdrant_retriever_sport, "question": itemgetter("question")}
            | rag_prompt | llm | StrOutputParser()
        )
        #
        rag_chain.invoke({"question" : "What does the 'context' in 'long context' refer to give me answer in sortest way?"})

        return f"[Medical RAG] For your question: '{question}', please consult a qualified doctor."

    @tool
    def movie_expert(query: str) -> str:
        """Answer movie-related questions."""

        
        return f"[Movie RAG] For your question: '{query}', check IMDb or Letterboxd."

    @tool
    def sports_expert(query: str) -> str:
        """Answer sports-related questions."""
        return f"[Sports RAG] For your question: '{query}', check ESPN or official league sites."
    



    tools = [medical_expert, movie_expert, sports_expert]
    llm_with_tools = llm.bind_tools(tools)

    # ---- Step 4: Define Nodes ----
    def chatbot(state: State):
        response = llm_with_tools.invoke(state["messages"])
        return {"messages": [response]}

    tool_node = ToolNode(tools=tools)

    # ---- Step 5: Build Graph ----
    graph_builder = StateGraph(State)
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", tool_node)
    graph_builder.add_edge(START, "chatbot")
    graph_builder.add_conditional_edges("chatbot", tools_condition)
    graph_builder.add_edge("tools", END)

    graph = graph_builder.compile()

    # ---- Step 6: Run Graph ----
    result = graph.invoke({
        "messages": [HumanMessage(content="question")]
    })

    return result["messages"][-1].content


In [ ]:
from typing import Annotated

from langchain.chat_models import init_chat_model
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


llm = llm


def sport(state: State):

    
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", sport)
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

In [31]:
print(graph.input_schema)

<class 'langgraph.utils.pydantic.LangGraphInput'>


In [33]:
graph.invoke({"input": "hello"})


BadRequestError: Error code: 400 - {'error': {'message': "'messages' : minimum number of items is 1", 'type': 'invalid_request_error'}}

In [35]:
output = graph.invoke({
    "messages": [
        {"role": "user", "content": "Hello!"}
    ]
})

# print(output["messages"][-1]["content"])  # Print assistant response


In [37]:
ai_reply = output["messages"][-1].content
print("🤖 Assistant:", ai_reply)


🤖 Assistant: <think>

</think>

Hello! How can I assist you today? 😊


In [ ]:
"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"

In [39]:
from dotenv import load_dotenv
load_dotenv() 

True

In [40]:
from langchain_tavily import TavilySearch

tool = TavilySearch(max_results=2)
tools = [tool]
tool.invoke("What's a 'node' in LangGraph?")

{'query': "What's a 'node' in LangGraph?",
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'LangGraph Tutorial: What Is LangGraph and How to Use It?',
   'url': 'https://www.datacamp.com/tutorial/langgraph-tutorial',
   'content': 'LangGraph is a library within the LangChain ecosystem that provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured and efficient manner. By managing the flow of data and the sequence of operations, LangGraph allows developers to focus on the high-level logic of their applications rather than the intricacies of agent coordination. Whether you need a chatbot that can handle various types of user requests or a multi-agent system that performs complex tasks, LangGraph provides the tools to build exactly what you need. LangGraph significantly simplifies the development of complex LLM applications by providing a structured framework for managing state and coordinating agent

In [41]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [42]:
# Modification: tell the LLM which tools it can call
# highlight-next-line
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

In [43]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

In [44]:
output = graph.invoke({
    "messages": [
        {"role": "user", "content": "what is langchain"}
    ]
})

In [45]:
ai_reply = output["messages"][-1].content
print("🤖 Assistant:", ai_reply)


🤖 Assistant: LangChain is an open-source orchestration framework designed for building applications with large language models (LLMs). It provides libraries in both Python and JavaScript, simplifying the development of AI-driven applications such as chatbots and AI agents. LangChain allows developers to chain together modular components, reducing the amount of code needed and making it easier to execute complex natural language processing (NLP) tasks. It supports the creation of sophisticated AI applications by connecting and orchestrating various language models and agents, enabling seamless data access, transformation, and sharing. Additionally, LangChain offers LangGraph, a framework for multi-agent orchestration, allowing different AI agents to interact and collaborate. This tool is particularly useful for developers looking to create advanced AI-driven solutions without needing extensive expertise in each underlying model.


In [ ]:
from typing import Annotated
from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage, AIMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# Define the State
class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

# Build the graph
graph_builder = StateGraph(State)

# --- LLM and Tool Setup ---
llm = llm

tool = TavilySearch(max_results=2)
tools = [tool]

# LLM with tools bound
llm_with_tools = llm.bind_tools(tools)

# --- Nodes ---

# Chatbot node
def chatbot(state: State):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

graph_builder.add_node("chatbot", chatbot)

# Tool node (LangGraph built-in)
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

# Summarizer node
def summarizer(state: State):
    last_message = state["messages"][-1].content
    summary = "Summary: " + last_message[:100]
    return {"messages": [AIMessage(content=summary)]}

graph_builder.add_node("summarizer", summarizer)

# --- Edges ---

# START → chatbot
graph_builder.add_edge(START, "chatbot")

# chatbot → tools or END (based on tool condition)
graph_builder.add_conditional_edges("chatbot", tools_condition)

# tools → summarizer → END
graph_builder.add_edge("tools", "summarizer")
graph_builder.add_edge("summarizer", END)

# Compile the graph
graph = graph_builder.compile()


In [ ]:
from typing import Annotated, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from typing_extensions import TypedDict
from langchain.chat_models import ChatOpenAI
from langchain_core.tools import tool

# -------- State Definition --------
class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

# -------- Base LLM --------
llm = llm

# -------- Define Custom Tools --------

@tool
def medical_expert_tool(query: str) -> str:
    """Answer questions related to medical health, disease, and symptoms."""
    return f"I'm a medical assistant. You asked: {query}"

@tool
def movie_expert_tool(query: str) -> str:
    """Answer questions about movies, actors, or film industry."""
    return f"I'm a movie buff. You asked: {query}"

@tool
def sports_expert_tool(query: str) -> str:
    """Answer questions about sports, teams, players, and scores."""
    return f"I'm a sports analyst. You asked: {query}"

# Bundle tools
tools = [medical_expert_tool, movie_expert_tool, sports_expert_tool]

# Bind LLM with tools
llm_with_tools = llm.bind_tools(tools)

# -------- Define Chat Node --------
def main_chatbot(state: State):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# -------- Tool Node --------
tool_node = ToolNode(tools=tools)

# -------- Create Graph --------
graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", main_chatbot)
graph_builder.add_node("tools", tool_node)

# Routing
graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", END)

graph = graph_builder.compile()

# -------- Run It --------
if __name__ == "__main__":
    queries = [
        "What are the symptoms of dengue?",
        "Who acted in Interstellar?",
        "Who won the FIFA World Cup in 2022?"
    ]

    for q in queries:
        print(f"\nUser: {q}")
        result = graph.invoke({"messages": [HumanMessage(content=q)]})
        print("Bot:", result["messages"][-1].content)
